In [ ]:
! pip install -q praw


     |████████████████████████████████| 176 kB 4.4 MB/s 
     |████████████████████████████████| 54 kB 1.4 MB/s 


In [ ]:
import praw
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from nltk.stem import WordNetLemmatizer
from collections import Counter
lemmatizer = WordNetLemmatizer()

%matplotlib inline


## Scraping Reddit

This reddit variable takes our Reddit API credentials to validate us when we query the API. From there we pull the information we need into panda dataframes.

In [ ]:
reddit = praw.Reddit(client_id='w1MJIPXRvmlQ7MRARzCGnw', client_secret='KOkGIZ61KBaN5QuX_zknle2qXcOLEA', user_agent='Andrea Gutierrez', check_for_async=False )

In [ ]:
# Query the 20 top posts from the Liberal subreddit and create a dataframe with the 
# title ID, title, subreddit, body, and number of total comments
_posts = []
lib_subreddit = reddit.subreddit('Liberal')
for post in lib_subreddit.top(limit=20):
    l_posts.append([post.id, post.title, post.subreddit, post.num_comments, post.selftext])
l_posts = pd.DataFrame(l_posts,columns=['id', 'title', 'subreddit', 'num_comments', 'body']) 
l_posts.head()

,id,title,subreddit,num_comments,body
0,6793yx,More GOP politicians have been arrested for se...,Liberal,135,
1,5wnt2h,Trump’s pledge to “start winning wars again” i...,Liberal,218,
2,6eepc1,Jared Kushner 'admitted Donald Trump lies to h...,Liberal,251,
3,758bq6,Chris Murphy: Pence walkout 'a multi million d...,Liberal,105,
4,8d90nb,"Stormy Daniels to Donate $130,000 to Planned P...",Liberal,142,


In [ ]:
# Query the 20 top posts from the Conservative subreddit and create a dataframe with the 
# title ID, title, subreddit, body, and number of total comments
c_posts = []
con_subreddit = reddit.subreddit('Conservative')
for post in con_subreddit.top(limit=20):
    c_posts.append([post.id, post.title, post.subreddit, post.num_comments, post.selftext])
c_posts = pd.DataFrame(c_posts,columns=['id', 'title', 'subreddit', 'num_comments', 'body'])
c_posts.head()

,id,title,subreddit,num_comments,body
0,krxl6t,For those of you comparing these protests to B...,Conservative,7849,
1,jpu2rx,AP Has called the election for Joe Biden,Conservative,9796,
2,kibtqm,I want my taxes back,Conservative,5188,
3,kw2lq8,Fox News: McConnell believes Trump committed i...,Conservative,6748,
4,ftaazt,China is paying reddit to take down pictures o...,Conservative,1295,


In [ ]:
# Lib_ID gathers each unique ID from every subreddit post and stores it in an array.
lib_id = []
lib_comment = reddit.subreddit('Liberal')
for post in lib_comment.top(limit=20):
    lib_id.append(reddit.submission(post.id))

# This code block appends the unique post ID to each comment within said post.
i = 0
lib_comments = []
for top_level_comment in lib_id:
    top_level_comment = lib_id[i]
    submission = reddit.submission(id = top_level_comment)
    submission.comments.replace_more(limit = 0)
    i += 1
    for c in submission.comments.list():
      lib_comments.append([c.body, submission.id])
      
lib_comments = pd.DataFrame(lib_comments, columns = ['comment', "id"])
lib_comments.shape

(2190, 2)

In [ ]:
# Cons_ID gathers each unique ID from every subreddit post and stores it in an array.
cons_id = []
cons_comment = reddit.subreddit('Conservative')
for post in cons_comment.top(limit=20):
    cons_id.append(reddit.submission(post.id))

# This code block appends the unique post ID to each comment within said post.
i = 0
cons_comments = []
for top_level_comment in cons_id:
    top_level_comment = cons_id[i]
    submission = reddit.submission(id = top_level_comment)
    submission.comments.replace_more(limit = 1)
    i += 1
    for n in submission.comments:
      cons_comments.append([n.body, submission.id])
      
cons_comments = pd.DataFrame(cons_comments, columns = ['comment','id'])
cons_comments.shape

(2874, 2)

## Cleaning Reddit Data

Here we clean and re-format our incoming Reddit data and confirm that our two subreddits are distinct enough to provide an interesting model.

In [ ]:
#cons_comments, lib_comments, c_posts, l_posts  (comments can be indexed from post)

#cl_posts = pd.merge(left = c_posts, right = l_posts, left_on = "id", right_on = "id", how = "outer")
array = [l_posts, c_posts]
cl_posts = pd.concat(array)
df = pd.DataFrame (cl_posts, columns=['id', 'title', 'subreddit', 'num_comments', 'body'])
df


def give_binary_values(dataset,col,one,zero): 
    dataset[col].loc[dataset[col]==one] = int(1)
    dataset[col].loc[dataset[col]==zero] = int(0)
    return dataset

give_binary_values(df,"subreddit","conservative", "liberal")

array2 = [lib_comments, cons_comments]
cl_comms = pd.concat(array2)
cl_comms

final = pd.merge(left = df, right = cl_comms, left_on = "id", right_on = "id", how = "right")
final['title'] = final['title'].str.lower()
final['body'] = final['body'].str.lower()
final['comment'] = final['comment'].str.lower()
final

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,id,title,subreddit,num_comments,body,comment
0,6793yx,more gop politicians have been arrested for se...,0,135,,i guess that makes sense. they see all the fuc...
1,6793yx,more gop politicians have been arrested for se...,0,135,,time for the gop in nc and other states to wri...
2,6793yx,more gop politicians have been arrested for se...,0,135,,larry craig will forever be the face of gop se...
3,6793yx,more gop politicians have been arrested for se...,0,135,,i have had a few conversations with conservati...
4,6793yx,more gop politicians have been arrested for se...,0,135,,seems like it would be a wise decision to ban ...
...,...,...,...,...,...,...
5059,kiif8x,breaking news: donald trump threatens to veto ...,1,5266,,it makes me so happy to see both sides united ...
5060,kiif8x,breaking news: donald trump threatens to veto ...,1,5266,,"what is happening?!\n\n left is right, up is d..."
5061,kiif8x,breaking news: donald trump threatens to veto ...,1,5266,,"although i think $2k would be more helpful, a ..."
5062,kiif8x,breaking news: donald trump threatens to veto ...,1,5266,,why isn't this front page on r/news or r/world...


In [ ]:
conservative_df = final[final['subreddit'] == 1]
conservative_df = conservative_df.drop(['id', 'title', 'num_comments', 'body', 'comment_token', 'body_token', 'title_token'], axis = 1)
liberal_df = final[final['subreddit'] == 0]
liberal_df = liberal_df.drop(['id', 'title', 'num_comments', 'body', 'comment_token', 'body_token', 'title_token'], axis = 1)

In [ ]:
lib = list(liberal_df['comment'])
con = list(conservative_df['comment'])

string1=""
for i in lib:
   string1=string1+i

string2=""
for i in con:
   string2=string2+i

# define a tokenize function using regex based on whitespace (and remove apostrophes)
def regtokenize(the_string):
    """Convert string to list of words"""
    the_string = re.sub(r"[\.\?!,;'’]", r"", the_string)
    return re.findall(r"\w+", the_string)

#define a bigram function
def bigrammer(input_list):
    bigram_list = []
    for i in range(len(input_list)-1):
        bigram_list.append((input_list[i], input_list[i+1]))
    return bigram_list

#applying regex tokenizer, counter, bigrammer, printing to test

lib_tokenized = regtokenize(string1)
print(lib_tokenized [:20])

con_tokenized = regtokenize(string2)
print(con_tokenized [:20])

#cutting off the tokenzied conservative list so that they both have the same number of words
con_tokenized = con_tokenized [:63185]


print()


lib_counter = Counter(lib_tokenized)
print(lib_counter.most_common())

con_counter = Counter(con_tokenized)
print(con_counter.most_common())


print()


lib_bigrams = bigrammer(lib_tokenized)
print(lib_bigrams[:100])

con_bigrams = bigrammer(con_tokenized)
print(con_bigrams[:100])


print()


#getting bigram counts
lib_bgcounts = Counter(lib_bigrams)
print(lib_bgcounts)

#getting bigram counts
con_bgcounts = Counter(con_bigrams)
print(con_bgcounts)

['i', 'guess', 'that', 'makes', 'sense', 'they', 'see', 'all', 'the', 'fucked', 'up', 'shit', 'that', 'goes', 'on', 'in', 'bathrooms', 'so', 'they', 'feel']
['tired', 'of', 'reporting', 'this', 'thread', 'debate', 'us', 'on', 'discord', 'instead', 'https', 'discordgg', 'conservative', 'this', 'is', 'an', 'automated', 'message', 'that', 'appears']

[('the', 2829), ('to', 1742), ('a', 1506), ('and', 1241), ('of', 1230), ('that', 962), ('is', 939), ('in', 862), ('it', 714), ('you', 692), ('i', 673), ('for', 634), ('are', 512), ('he', 468), ('not', 467), ('be', 433), ('this', 433), ('have', 426), ('was', 413), ('but', 402), ('they', 398), ('people', 394), ('on', 381), ('as', 330), ('with', 328), ('we', 304), ('if', 296), ('its', 288), ('about', 280), ('just', 269), ('all', 267), ('trump', 266), ('or', 258), ('dont', 252), ('so', 248), ('would', 243), ('like', 238), ('his', 236), ('what', 231), ('who', 221), ('no', 207), ('from', 205), ('at', 201), ('their', 197), ('more', 196), ('do', 195)

Above we can start to see how the two corpora begin to differ when counting our bigrams. This tells us (very roughly) that our classification models should be able to differentiate between the two. We were worried there might not be a noticable difference, which would lead us to pick two different subreddits to classify. However, at this point it looks like there is no need to change the two subreddits we're classifying just yet -- we're good to move forward. 

In [ ]:
a = ['subreddit', 'comment']
final_df = final[a]

#Drop duplicate entries in our comments and check shape before and after
final_df.drop_duplicates(keep='first', inplace=True)
print(final.shape)
print(final_df.shape) # It can be seen that some duplicates were dropped

#Checking our distribution of the two subreddits
final_df.subreddit.value_counts(normalize = True)

(5064, 9)
(4612, 2)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


1    0.562446
0    0.437554
Name: subreddit, dtype: float64

We can see that our subreddit distribution still leans on the Convervative side - we'll want to keep this in mind as we instantiate our models. 

In [ ]:
#Confirm no null values
final_df.isnull().sum()

subreddit    0
comment      0
dtype: int64

##Modelling

Here we take our subreddits and classify them across 4 different model methods, each with two different parameterized tests, to see how well we can classify and which model performs most accurately. 

### Establishing Features

In [ ]:
# creating a features list and setting our X equal to it
features = ['comment']
X = final_df[features]

# setting our Y equal to our target column
y = final_df.subreddit


# ended up dropping titles to avoid duplicates in our model - it was causing our models to report perfect classification because of these duplicates.

In [ ]:
# instantiating our train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42, # setting a random state for reproducibility
                                                    test_size=0.33,
                                                    stratify=y) # stratifying our Y because the two corpora are slightly unbalanced

# Explicitly setting y to an int type (model was throwing errors without this being explicitly typed)                                                    
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Validating our split
print('Number of records in the original dataset: ', len(y))
print('Number of records in the training dataset: ', len(y_train))
print('Number of records in the testing dataset: ', len(y_test))

Number of records in the original dataset:  4612
Number of records in the training dataset:  3090
Number of records in the testing dataset:  1522


In [ ]:
# instantiating a CountVectorizer on our subreddit comment text, removing english stop words, and stripping ascii accents
# the assumption was that subreddit comments with an ngram range of (1, 5) would be more helpful than those with a smaller range of say (1, 2)
# we set a min_df to ignore terms that appear less than 2% of the time in our corpus
cv_text = CountVectorizer(stop_words='english', strip_accents = 'ascii', ngram_range=(1, 5), min_df=.02)

In [ ]:
# this should give us two difference matrices, one for text test and train
X_train_text = cv_text.fit_transform(X_train.comment)
X_test_text = cv_text.transform(X_test.comment)

In [ ]:
# creating a dataframe with our train subreddit text and checking the shape
X_train_text_df = pd.DataFrame(X_train_text.todense(), columns=[x+'_text' for x in cv_text.get_feature_names()])
print(X_train_text_df.shape)

(3090, 63)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# creating a dataframe with our test subreddit text and checking the shape
X_test_text_df = pd.DataFrame(X_test_text.todense(), columns=[x+'_text' for x in cv_text.get_feature_names()])
print(X_test_text_df.shape)

(1522, 63)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# double-checking to make sure there are no null values
print(X_train_text_df.isnull().sum().sum())
print(X_test_text_df.isnull().sum().sum())

0
0


In [ ]:
#instantiate the model
trial = LogisticRegression()

#fit the model using the training data
trial.fit(X_train_text_df,y_train)

#use model to make predictions on test data
y_pred = trial.predict(X_test_text_df)

quick_r2 = trial.score(X_train_text_df, y_train)
print("R^2: {0}".format(quick_r2))
#Quickly checking R^2 to make sure our duplicate problem is not repeating itself - looks good to try our models again! 

R^2: 0.63915857605178


For the models below we find the two best parameters for each model type and:

Fit the models

Print the train and test scores for the models

Make predictions and print them in a confusion matrix

Print out the best parameters

### Logistic Regression

In [ ]:
def run_the_lr_models(model):
    
    if model == 'lr_1':
        
        lr_1_params = {
            'penalty': ['l1'],
            'C': [1, 1.5, 2, 2.5],
            'class_weight': ['balanced'],
            'warm_start': [True, False],
            'random_state': [42],
            'solver': ['liblinear']}
        
        M = GridSearchCV(LogisticRegression(),
                        lr_1_params,
                        cv = 5,
                        verbose = 1,
                        n_jobs = -1)
        
    elif model == 'lr_2':
        
        lr_2_params = {
            'penalty': ['l2'],
            'C': [1, 1.5, 2, 2.5],
            'class_weight': ['balanced'],
            'warm_start': [True, False],            
            'random_state': [42],
            'solver': ['lbfgs', 'liblinear']}
        
        M = GridSearchCV(LogisticRegression(),
                        lr_2_params,
                        cv = 5,
                        verbose = 1,
                        n_jobs = -1)

    else:
        print('There is an error.')
        
    M.fit(X_train_text_df.values, y_train)
     
    print(f'Train score = {M.score(X_train_text_df.values, y_train)}')
    print(f'Test score = {M.score(X_test_text_df.values, y_test)}')
    
    predictions = M.predict(X_test_text_df.values)
    print('--------')
    print(confusion_matrix(y_test, predictions))
    print(f'Best params = {M.best_params_}')

In [ ]:
run_the_lr_models('lr_1')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Train score = 0.6220064724919094
Test score = 0.5814717477003942
--------
[[494 172]
 [465 391]]
Best params = {'C': 2.5, 'class_weight': 'balanced', 'penalty': 'l1', 'random_state': 42, 'solver': 'liblinear', 'warm_start': True}


In [ ]:
run_the_lr_models('lr_2')

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Train score = 0.6200647249190938
Test score = 0.5827858081471747
--------
[[491 175]
 [460 396]]
Best params = {'C': 1, 'class_weight': 'balanced', 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'warm_start': True}


The difference between these two models are the penalty (aka lassa vs ridge regression). Some parameters only worked with an L1 penalty and vice versa, so they were re-parameterized as needed.

### Decision Tree Models


In [ ]:
def run_the_dt_models(model):
    
    if model == 'dt_1':
        
        dt_params = {
            'criterion': ['gini'],
            'max_depth': [4, 24, 54],
            'min_samples_split': [5, 7, 11, 14],
            'max_features': [None, 'log2', 'auto', .40, .50, .70],
            'random_state': [42]}
        
        M = GridSearchCV(DecisionTreeClassifier(),
                        dt_params,
                        cv = 5,
                        verbose = 1,
                        n_jobs = -1)
   
    elif model == 'dt_2':
        
        dt_none_params = {
            'criterion': ['entropy'],
            'max_depth': [4, 24, 54],
            'min_samples_split': [5, 7, 11, 14],
            'max_features': [None, 'log2', 'auto', .40, .50, .70],
            'random_state': [42]}
        
        M = GridSearchCV(DecisionTreeClassifier(),
                        dt_none_params,
                        cv = 5,
                        verbose = 1,
                        n_jobs = -1)

    else:
        print('There is an error.')
        
    M.fit(X_train_text_df.values, y_train)
     
    print(f'Train score = {M.score(X_train_text_df.values, y_train)}')
    print(f'Test score = {M.score(X_test_text_df.values, y_test)}')
    
    predictions = M.predict(X_test_text_df.values)
    print('--------')
    print(confusion_matrix(y_test, predictions))
    print(f'Best params = {M.best_params_}')

In [ ]:
run_the_dt_models('dt_1')

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Train score = 0.7378640776699029
Test score = 0.5965834428383706
--------
[[238 428]
 [186 670]]
Best params = {'criterion': 'gini', 'max_depth': 54, 'max_features': None, 'min_samples_split': 14, 'random_state': 42}


In [ ]:
run_the_dt_models('dt_2')

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Train score = 0.7268608414239482
Test score = 0.6051248357424441
--------
[[225 441]
 [160 696]]
Best params = {'criterion': 'entropy', 'max_depth': 54, 'max_features': 'auto', 'min_samples_split': 14, 'random_state': 42}


The difference between these two models are Gini (dt_1) and Entropy (dt_2).

### Random Forest Models

In [ ]:

def run_the_rf_models(model):
    
    if model == 'rf_1':
        #theoretic information 
        rf_params = {
            'n_estimators': [15, 24, 30],
            'criterion': ['gini'],
            'max_depth': [None, 5, 13, 21],
            'bootstrap': [True, False],
            'min_samples_split': [5, 7, 15, 25],
            'max_features': [None, 'log2', 'auto', .10, .25, .50],
            'warm_start': [True],
            'random_state': [42]}
        
        M = GridSearchCV(RandomForestClassifier(),
                        rf_params,
                        cv = 5,
                        verbose = 1,
                        n_jobs = -1)
        
    elif model == 'rf_2':
        #information theoretic estimate Looking at relationships between data
        rf_none_params = {
            'n_estimators': [15, 24, 30],
            'criterion': ['entropy'],
            'max_depth': [None, 5, 13, 21],
            'bootstrap': [True, False],
            'min_samples_split': [5, 7, 15, 25],
            'max_features': [None, 'log2', 'auto',  .10, .25, .50],
            'warm_start': [True],
            'random_state': [42]}
        
        M = GridSearchCV(RandomForestClassifier(),
                        rf_none_params,
                        cv = 5,
                        verbose = 1,
                        n_jobs = -1)
        
    else:
        print('There is an error.')
        
    M.fit(X_train_text_df.values, y_train)
     
    print(f'Train score = {M.score(X_train_text_df.values, y_train)}')
    print(f'Test score = {M.score(X_test_text_df.values, y_test)}')
    
    predictions = M.predict(X_test_text_df.values)
    print('--------')
    print(confusion_matrix(y_test, predictions))
    print(f'Best params = {M.best_params_}')

In [ ]:
run_the_rf_models('rf_1')

Fitting 5 folds for each of 576 candidates, totalling 2880 fits
Train score = 0.7194174757281553
Test score = 0.6248357424441524
--------
[[197 469]
 [102 754]]
Best params = {'bootstrap': True, 'criterion': 'gini', 'max_depth': 21, 'max_features': 'auto', 'min_samples_split': 7, 'n_estimators': 24, 'random_state': 42, 'warm_start': True}


In [ ]:
run_the_rf_models('rf_2')

Fitting 5 folds for each of 576 candidates, totalling 2880 fits
Train score = 0.7417475728155339
Test score = 0.6222076215505913
--------
[[246 420]
 [155 701]]
Best params = {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'min_samples_split': 25, 'n_estimators': 30, 'random_state': 42, 'warm_start': True}


The difference between these two models are Gini (rf_1) and Entropy (rf_2).

### Multinomial Naive Bayes Models

In [ ]:
def run_the_mn_models(model):
    
    if model == 'mn_1':
        
        mn_params = {
            'fit_prior': [True],
            'alpha': [0, 0.5, 1]}
        
        M = GridSearchCV(MultinomialNB(),
                        mn_params,
                        cv = 5,
                        verbose = 1,
                        n_jobs = -1)
        
    elif model == 'mn_2':
        
         mn_params = {
            'fit_prior': [False],
            'alpha': [0, 0.5, 1]}
        
         M = GridSearchCV(MultinomialNB(),
                        mn_params,
                        cv = 5,
                        verbose = 1,
                        n_jobs = -1)
        
    else:
        print('There is an error.')
        
    M.fit(X_train_text_df.values, y_train)
     
    print(f'Train score = {M.score(X_train_text_df.values, y_train)}')
    print(f'Test score = {M.score(X_test_text_df.values, y_test)}')
    
    predictions = M.predict(X_test_text_df.values)
    print('--------')
    print(confusion_matrix(y_test, predictions))
    print(f'Best params = {M.best_params_}')

In [ ]:
run_the_mn_models('mn_1')

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Train score = 0.6394822006472491
Test score = 0.6228646517739816
--------
[[240 426]
 [148 708]]
Best params = {'alpha': 0, 'fit_prior': True}


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


In [ ]:
run_the_mn_models('mn_2')

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Train score = 0.6174757281553398
Test score = 0.5900131406044679
--------
[[477 189]
 [435 421]]
Best params = {'alpha': 0, 'fit_prior': False}


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


The difference between the two models here lie in the fit_prior argument, with one model having it set equal to True (mn_1) and the other to False (mn_2).